In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
import os
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [4]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

In [5]:
llm.invoke("hi hello how are you?")

AIMessage(content="Hello! I'm just a computer program so I don't have feelings, but I'm here and ready to help. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 13, 'total_tokens': 44, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Bl55GvK4jcVGnj3RXyiegEMUPnFjW', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--8dbf8bcb-942f-4efb-89ea-1f7d627a4fb7-0', usage_metadata={'input_tokens': 13, 'output_tokens': 31, 'total_tokens': 44, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [6]:
from langgraph.types import Command

In [7]:
from langgraph.prebuilt import create_react_agent

In [ ]:
# class AgentState(TypedDict):
#     messages: Annotated[Sequence[BaseMessage], add_messages]
#     Name:str
#     age:int
#     DOB:int

In [8]:
def add_number(state):
    result=state["num1"]+state["num2"]
    print(f"addition is {result}")
    return Command(goto="multiply",update={"sum":result})

In [9]:
state={"num1":10,"num2":20}

In [10]:
add_number(state)

addition is 30


Command(update={'sum': 30}, goto='multiply')

### Creating one dummy multiagent

it is for network/collab multiagent

In [12]:
from langchain_core.tools import tool

In [13]:
@tool
def transfer_to_multiplication_expert():
    """Ask multiplication agent for help"""
    return

In [14]:
@tool
def transfer_to_addition_expert():
    """Ask addition agent for help"""
    return

In [15]:
llm_with_tool=llm.bind_tools([transfer_to_multiplication_expert])

In [16]:
response=llm_with_tool.invoke("hi")

In [17]:
response.content

'Hello! How can I assist you today?'

In [18]:
response.tool_calls

[]

In [19]:
response=llm_with_tool.invoke("what is 2 multiply 2?")

In [20]:
response.content

''

In [21]:
response.tool_calls

[{'name': 'transfer_to_multiplication_expert',
  'args': {},
  'id': 'call_OtB8gzmR7XBlsyEFIYYrFkr7',
  'type': 'tool_call'}]

In [29]:
from typing_extensions import Literal
from langgraph.graph import MessagesState,StateGraph, START,END

In [ ]:
def additional_expert(state:MessagesState)-> Command[Literal["multiplication_expert", "__end__"]]:
    pass

In [ ]:
def multiplication_expert(state:MessagesState)-> Command[Literal["additional_expert", "__end__"]]:
    pass

In [25]:
graph=StateGraph(MessagesState)

In [26]:
graph.add_node("addition_agent",additional_expert)

In [27]:
graph.add_node("muliplication_agent",multiplication_expert)

In [30]:
graph.add_edge(START,"addition_agent")

In [31]:
app=graph.compile()

ValueError: Found edge ending at unknown node `additional_expert`